In [1]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:86% !important;}
div.cell.code_cell.rendered{width:100%;}
div.CodeMirror {font-family:Consolas; font-size:12pt;}
div.output {font-size:15pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:12pt;}
div.prompt {min-width:70px;}
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{font-size:12pt;padding:5px;}
table.dataframe{font-size:15px;}
</style>
"""))

# DB에서 과거 정보 로드

In [ ]:
def DBMS_find_1year_ago_of_tomorrow():
    """
    오늘 기준으로 '내일'의 1년 전(같은 시각) 데이터를 DB에서 조회하는 함수
    """
    import pymysql
    from dotenv import load_dotenv
    import os
    import pandas as pd
    from datetime import datetime, timedelta

    load_dotenv()
    host = os.getenv('host')
    user = os.getenv('user')
    password = os.getenv('password')
    database = os.getenv('database')
    port = int(os.getenv('port', 3306))

    # 현재 시각
    now = datetime.now()

    # 내일 같은 시각
    tomorrow = now + timedelta(days=1)

    # 1년 전 내일 같은 시각
    dt_1y = tomorrow.replace(year=tomorrow.year - 1)

    # DB 컬럼명에 맞게 쿼리 작성 (연, 월, 일, 시)
    query =f"""
        SELECT *
        FROM basic_data
        WHERE year = {dt_1y.year}
          AND month = {dt_1y.month}
          AND day = {dt_1y.day}
          AND hour = {dt_1y.hour}"""

    try:
        connection = pymysql.connect(
            host=host,
            user=user,
            password=password,
            database=database,
            port=port,
            charset='utf8mb4',
            cursorclass=pymysql.cursors.DictCursor
        )
        with connection.cursor() as cursor:
            cursor.execute(query)
            data = cursor.fetchall()
            result_df = pd.DataFrame(data)
#             print(result_df)
            return result_df
    except Exception as e:
        print('에러 발생:', e)
        return None
    finally:
        try:
            connection.close()
        except:
            pass

# 사용법
DBMS_find_1year_ago_of_tomorrow()

In [2]:
def class_to_range(pred_class):
    # qcut 구간을 이용해서 문자로 반환
    bins = [1, 19, 37, 62, 93, 131, 180, 254, 393, 3258]
    pred_class = int(pred_class)
    left = int(bins[pred_class])
    right = int(bins[pred_class+1]) if pred_class+1 < len(bins) else '+'
    return f"{left} ~ {right}"

# 익일 같은 시간대 예측 함수

In [3]:
# 예측 함수
def nextday_predict_bike_demand(행정구, 강수, 습도, 풍속, 기온):
    """
    행정구명, 강수, 습도, 풍속, 기온을 입력받아
    해당 시간의 공공자전거 대여량 예측 클래스를 반환하는 함수
    """
    import pandas as pd
    import numpy as np
    import joblib
    from datetime import datetime, timedelta
    import holidays
    import tensorflow as tf

    le = joblib.load('ENCODER_SCALER/labelencoders.pkl')
    scaler = joblib.load('ENCODER_SCALER/scaler.pkl')
    feature_cols = joblib.load('ENCODER_SCALER/featurecols.pkl')
    num_cols = joblib.load('ENCODER_SCALER/numcols.pkl')
    model = tf.keras.models.load_model('best_lstm_model.h5')

    now = datetime.now()
    년, 월, 일, 시 = now.year, now.month, now.day, now.hour
    요일 = now.weekday()
    주말구분 = 1 if 요일 >= 5 else 0
    kr_holidays = holidays.KR(years=[년])
    공휴일 = int(now.date() in kr_holidays)
    df_1y = DBMS_find_1year_ago_of_tomorrow()
    대여량_1년전 = df_1y[df_1y['district']==행정구].loc[:,'rental_count']
    총생활인구수_1년전 = df_1y[df_1y['district']==행정구].loc[:,'total_population']

    input_dict = {
        '행정구': le.transform([행정구])[0],
        '월': 월,
        '일': 일,
        '시': 시,
        '요일': 요일,
        '강수': 강수,
        '기온': 기온,
        '습도': 습도,
        '풍속': 풍속,
        '주말구분': 주말구분,
        '공휴일': 공휴일,
        '대여량_1년전': 대여량_1년전,
        '총생활인구수_1년전': 총생활인구수_1년전,
    }
    input_df = pd.DataFrame([input_dict])
    input_df[num_cols] = scaler.transform(input_df[num_cols])
    X_input = input_df[feature_cols].values
    X_input = np.expand_dims(X_input, axis=1)
    pred = model.predict(X_input,verbose=0)
    pred_class = int(np.argmax(pred, axis=1)[0])
    return pred_class

In [4]:
predict_bike_demand('강남구',9.5, 70,9.2,30) # (행정구, 강수, 습도, 풍속, 기온)

7

In [4]:
# from tqdm import tqdm # 진행율 progress bar 표시

# for i in tqdm(range(100)):
#     # 무언가 작업
#     pass

# 시간대별 행정구별 기온 강수 습도 풍속 함수

In [41]:
import requests
import pandas as pd

def get_hourly_weather_seoul_openmeteo():
    SEOUL_DISTRICTS = [
        {"name": "강남구", "lat": 37.5172, "lon": 127.0473},
        {"name": "강동구", "lat": 37.5301, "lon": 127.1238},
        {"name": "강북구", "lat": 37.6396, "lon": 127.0256},
        {"name": "강서구", "lat": 37.5509, "lon": 126.8495},
        {"name": "관악구", "lat": 37.4784, "lon": 126.9516},
        {"name": "광진구", "lat": 37.5384, "lon": 127.0823},
        {"name": "구로구", "lat": 37.4955, "lon": 126.8878},
        {"name": "금천구", "lat": 37.4604, "lon": 126.9006},
        {"name": "노원구", "lat": 37.6542, "lon": 127.0568},
        {"name": "도봉구", "lat": 37.6688, "lon": 127.0472},
        {"name": "동대문구", "lat": 37.5744, "lon": 127.0396},
        {"name": "동작구", "lat": 37.5124, "lon": 126.9392},
        {"name": "마포구", "lat": 37.5663, "lon": 126.9014},
        {"name": "서대문구", "lat": 37.5791, "lon": 126.9368},
        {"name": "서초구", "lat": 37.4836, "lon": 127.0327},
        {"name": "성동구", "lat": 37.5633, "lon": 127.0364},
        {"name": "성북구", "lat": 37.5894, "lon": 127.0167},
        {"name": "송파구", "lat": 37.5146, "lon": 127.1056},
        {"name": "양천구", "lat": 37.5179, "lon": 126.8666},
        {"name": "영등포구", "lat": 37.5264, "lon": 126.8962},
        {"name": "용산구", "lat": 37.5323, "lon": 126.9909},
        {"name": "은평구", "lat": 37.6027, "lon": 126.9291},
        {"name": "종로구", "lat": 37.5735, "lon": 126.9790},
        {"name": "중구", "lat": 37.5636, "lon": 126.9977},
        {"name": "중랑구", "lat": 37.6065, "lon": 127.0927}
    ]
    result_rows = []
    for gu in SEOUL_DISTRICTS:
        latitude, longitude, name = gu["lat"], gu["lon"], gu["name"]
        url = (
            f"https://api.open-meteo.com/v1/forecast"
            f"?latitude={latitude}&longitude={longitude}"
            "&hourly=temperature_2m,precipitation,weathercode,relative_humidity_2m,wind_speed_10m"
            "&forecast_days=7&timezone=Asia%2FSeoul"
        )
        resp = requests.get(url)
        if resp.status_code == 200:
            json_data = resp.json()
            # 여기에 'hourly' 키가 반드시 있어야 함
            if 'hourly' not in json_data:
                print(f"[경고] {name} 데이터에 'hourly' 없음. 응답: {json_data}")
                continue
            data = json_data['hourly']
            for t, temp, rain, code, rh, wind in zip(
                data['time'],
                data['temperature_2m'],
                data['precipitation'],
                data['weathercode'],
                data['relative_humidity_2m'],
                data['wind_speed_10m'],
            ):
                result_rows.append({
                    "구": name,
                    "일시": t,
                    "기온(℃)": temp,
                    "강수량(mm)": rain,
                    "날씨코드": code,
                    "습도(%)": rh,
                    "풍속(m/s)": wind,
                })
        else:
            print(f"[오류] {name} API 호출 실패: {resp.status_code}")
    df = pd.DataFrame(result_rows)
    return df

# 사용 예시
df = get_hourly_weather_seoul_openmeteo()
print(df.head(10))

     구                일시  기온(℃)  강수량(mm)  날씨코드  습도(%)  풍속(m/s)
0  강남구  2025-07-01T00:00   24.1      0.0     1     96      5.7
1  강남구  2025-07-01T01:00   24.0      0.0     1     96      5.9
2  강남구  2025-07-01T02:00   24.0      0.0     1     96      6.6
3  강남구  2025-07-01T03:00   23.9      0.0     1     96      7.2
4  강남구  2025-07-01T04:00   23.9      0.0     1     96      7.1
5  강남구  2025-07-01T05:00   23.8      0.0     1     96      7.6
6  강남구  2025-07-01T06:00   23.9      0.0     1     96      6.7
7  강남구  2025-07-01T07:00   24.4      0.0     2     94      7.2
8  강남구  2025-07-01T08:00   25.2      0.0     1     92      7.7
9  강남구  2025-07-01T09:00   25.8      0.0     3     91      7.4


In [42]:
get_hourly_weather_seoul_openmeteo()

,구,일시,기온(℃),강수량(mm),날씨코드,습도(%),풍속(m/s)
0,강남구,2025-07-01T00:00,24.1,0.0,1,96,5.7
1,강남구,2025-07-01T01:00,24.0,0.0,1,96,5.9
2,강남구,2025-07-01T02:00,24.0,0.0,1,96,6.6
3,강남구,2025-07-01T03:00,23.9,0.0,1,96,7.2
4,강남구,2025-07-01T04:00,23.9,0.0,1,96,7.1
...,...,...,...,...,...,...,...
4195,중랑구,2025-07-07T19:00,31.0,0.0,0,61,6.6
4196,중랑구,2025-07-07T20:00,29.1,0.0,0,73,5.4
4197,중랑구,2025-07-07T21:00,27.4,0.0,0,82,4.3
4198,중랑구,2025-07-07T22:00,26.2,0.0,0,87,3.6


# 1주일치 예상 대여량 예측하기 => DB 키고 체크해보기

In [43]:
import pandas as pd
import numpy as np
import joblib
from datetime import datetime, timedelta
import holidays
import tensorflow as tf

def one_week_predict_bike_demand():
    """
    25개 행정구별로 1주일(7일)간 시간대별 대여량 예측(class)을 반환합니다.

    - 7일(168시간) x 25개 구 = 4,200 예측 결과
    - get_hourly_weather_seoul_openmeteo()로 날씨 데이터 자동 추출(기온, 강수, 습도, 풍속)
    - holiday/주말/1년전대여량 등 feature 자동 생성
    - 결과: DataFrame (컬럼: 행정구, 날짜, 시각, 예측 class)
    """
    le = joblib.load('ENCODER_SCALER/labelencoders.pkl')
    scaler = joblib.load('ENCODER_SCALER/scaler.pkl')
    feature_cols = joblib.load('ENCODER_SCALER/featurecols.pkl')
    num_cols = joblib.load('ENCODER_SCALER/numcols.pkl')
    model = tf.keras.models.load_model('best_lstm_model.h5')

    # 7일간 날짜 리스트 생성
    now = datetime.now()
    target_dates = [(now + timedelta(days=d)).date() for d in range(7)]
    kr_holidays = holidays.KR(years=[now.year, now.year+1])

    # 날씨 정보(시간별) 불러오기
    weather_df = get_hourly_weather_seoul_openmeteo()
    weather_df['일시'] = pd.to_datetime(weather_df['일시'])
    weather_df['날짜'] = weather_df['일시'].dt.date
    weather_df['시'] = weather_df['일시'].dt.hour

    df_1y = DBMS_find_1year_ago_of_tomorrow()

    results = []

    for gu in weather_df['구'].unique():
        gu_weather = weather_df[weather_df['구'] == gu]
        for target_date in target_dates:
            daily_weather = gu_weather[gu_weather['날짜'] == target_date]
            if daily_weather.empty:
                continue
            for _, row in daily_weather.iterrows():
                월 = row['일시'].month
                일 = row['일시'].day
                시 = row['시']
                요일 = row['일시'].weekday()
                주말구분 = int(요일 >= 5)
                공휴일 = int(row['날짜'] in kr_holidays)
                강수 = row['강수량(mm)']
                기온 = row['기온(℃)']
                습도 = row['습도(%)']
                풍속 = row['풍속(m/s)']
                행정구 = gu
                # 1년전 feature (구/월/일/시 등 조건으로 찾기)
                try:
                    prev = df_1y[
                        (df_1y['district'] == 행정구) &
                        (df_1y['월'] == 월) & (df_1y['일'] == 일) & (df_1y['시'] == 시)
                    ].iloc[0]
                    대여량_1년전 = prev['rental_count']
                    총생활인구수_1년전 = prev['total_population']
                except:
                    대여량_1년전 = 0
                    총생활인구수_1년전 = 0

                input_dict = {
                    '행정구': le.transform([행정구])[0],
                    '월': 월,
                    '일': 일,
                    '시': 시,
                    '요일': 요일,
                    '강수': 강수,
                    '기온': 기온,
                    '습도': 습도,
                    '풍속': 풍속,
                    '주말구분': 주말구분,
                    '공휴일': 공휴일,
                    '대여량_1년전': 대여량_1년전,
                    '총생활인구수_1년전': 총생활인구수_1년전,
                }
                input_df = pd.DataFrame([input_dict])
                input_df[num_cols] = scaler.transform(input_df[num_cols])
                X_input = input_df[feature_cols].values
                X_input = np.expand_dims(X_input, axis=1)
                pred = model.predict(X_input, verbose=0)
                pred_class = int(np.argmax(pred, axis=1)[0])
                results.append({
                    "행정구": 행정구, "날짜": row['날짜'], "시": 시, "예측 class": pred_class
                })

    result_df = pd.DataFrame(results)
    return result_df